In [31]:
import pandas as pd
import numpy as np
import os.path

In [32]:
df = pd.read_csv('/Users/phaniram/Downloads/AND_dataset/Dataset[WithFeatures]/AND_Features.csv')

In [33]:
from random import randint
#label "0" for same class "1" for different class
def Label():
    colms = ['fa1', 'fa2','fa3', 'fa4', 'fa5', 'fa6', 'fa7', 'fa8', 'fa9', 'fb1', 'fb2', 'fb3', 'fb4', 'fb5', 'fb6', 'fb7', 'fb8', 'fb9' ,'c']
    sameLabels = pd.DataFrame(columns=colms)
    diffLabels = pd.DataFrame(columns=colms)
    rowNum = 0
    for index, row in df.iterrows():
        curImageId = row['ImageId'][:4]
        sameWriter = df[df['ImageId'].str.contains(curImageId)]
        diffWriter = df[~df['ImageId'].str.contains(curImageId)]
        #print(len(diffWriter))
        #print(sameWriter)
    
        for i in range(0, len(sameWriter)):
            np_a = sameWriter.iloc[i].as_matrix()
            np_sameLabel = np.concatenate((np_a[2:], np_a[2:]), axis =0)
            np_sameLabel= np.append(np_sameLabel,0)
            sameLabels.loc[rowNum] = np_sameLabel
            randomIndex = randint(0, len(diffWriter)-1)
            np_d = diffWriter.iloc[randomIndex].as_matrix()
            np_diffLabel = np.concatenate((np_a[2:], np_d[2:]), axis =0)
            np_diffLabel = np.append(np_diffLabel,1)
            diffLabels.loc[rowNum] =  np_diffLabel
            rowNum = rowNum+1
            
            for j in range(i+1, len(sameWriter)):
                #print(np_a)
                np_b = sameWriter.iloc[j].as_matrix()
                #print(np_b)
                #print("i", i)
                #print("j", j)
                np_sameLabel = np.concatenate((np_a[2:], np_b[2:]), axis =0)
                np_sameLabel = np.append(np_sameLabel,0)
                sameLabels.loc[rowNum] = np_sameLabel
                randomIndex = randint(0, len(diffWriter)-1)
                np_d = diffWriter.iloc[randomIndex].as_matrix()
                np_diffLabel = np.concatenate((np_a[2:], np_d[2:]), axis =0)
                np_diffLabel = np.append(np_diffLabel,1)
                diffLabels.loc[rowNum] =  np_diffLabel
                rowNum = rowNum+1
                #break        
        #print(sameLabels)
        #print('diff labess')
        #print(diffLabels)
        #break
    return sameLabels, diffLabels
    
        
    

In [34]:
sameLabels, diffLabels = Label()

In [35]:
#patch for a bug in pgmpy library
#sameLabels.fa9 = 
#sameLabels['fa9']= sameLabels['fa9']-1

sameLabels['fb9'] = sameLabels['fb9']-1
diffLabels['fa9'] = diffLabels['fa9']-1
diffLabels['fb9'] = diffLabels['fb9']-1

In [36]:
sameLabels = sameLabels.sample(frac=1).reset_index(drop=True)
sameLabels = sameLabels.sample(frac=1).reset_index(drop=True)
diffLabels = diffLabels.sample(frac=1).reset_index(drop=True)
diffLabels = diffLabels.sample(frac=1).reset_index(drop=True)
#this will shuffle the dataframe

In [37]:
columns = ['Id','ImageId', 'fa1', 'fa2','fa3', 'fa4', 'fa5', 'fa6', 'fa7', 'fa8', 'fa9', 'fb1', 'fb2', 'fb3', 'fb4', 'fb5', 'fb6', 'fb7', 'fb8', 'fb9']

In [38]:

trainSameLabels = sameLabels[:int(len(sameLabels)*0.7)] 
#while(trainSameLabels['fa9'].max() != 1 or trainSameLabels['fb9'].max() != 1 or trainSameLabels['fa9'].min() != 0 or trainSameLabels['fb9'].min() != 0):
    #sameLabels = sameLabels.sample(frac=1).reset_index(drop=True)
    #trainSameLabels = sameLabels[:int(len(sameLabels)*0.7)]

#trainSameLabels = sameLabels

In [39]:
trainDiffLabels = diffLabels[:int(len(diffLabels)*0.7)]
#while(trainDiffLabels['fa9'].max() != 1 or trainDiffLabels['fb9'].max() != 1):
    #diffLabels = diffLabels.sample(frac=1).reset_index(drop=True)
    #trainDiffLabels = diffLabels[:int(len(sameLabels)*0.7)]
#trainDiffLabels = diffLabels

In [40]:
testSameLabels = sameLabels[int(len(sameLabels)*0.7):]
#testSameLabels = sameLabels[int(len(sameLabels)*1.0):]

In [41]:
#trainDiffLabels = diffLabels[:int(len(diffLabels)*0.7)]

In [42]:
testDiffLabels = diffLabels[int(len(diffLabels)*0.7) :]
#testDiffLabels = diffLabels[int(len(diffLabels)*1.0) :]

In [43]:
#type(trainSameLabels)
colms = ['fa1', 'fa2','fa3', 'fa4', 'fa5', 'fa6', 'fa7', 'fa8', 'fa9', 'fb1', 'fb2', 'fb3', 'fb4', 'fb5', 'fb6', 'fb7', 'fb8', 'fb9', 'c']
train_data=pd.DataFrame(columns=colms)
train_data= train_data.append([trainSameLabels, trainDiffLabels])

In [44]:
## now training is remaining

In [45]:
from pgmpy.models import BayesianModel
model1 = BayesianModel([('fa1', 'fb1'),('fa2', 'fb2'),('fa3', 'fb3'),('fa4', 'fb4'),('fa5', 'fb5'),('fa6', 'fb6'),('fa7', 'fb7'),('fa8', 'fb8'),('fa9', 'fb9')
                        ,('c', 'fa1'), ('c', 'fb1'),('c', 'fa2'),('c', 'fb2'),('c', 'fa3'),('c', 'fb3'),('c', 'fa4'),('c', 'fb4'),('c', 'fa5'),('c', 'fb5'),('c', 'fa6'),('c', 'fb6'),('c', 'fa7'),('c', 'fb7'),('c', 'fa8'),('c', 'fb8'),('c', 'fa9'),('c', 'fb9')])

In [46]:
model1.fit(train_data)

In [47]:
model1.get_cpds()

[<TabularCPD representing P(c:2) at 0x10c8da2b0>,
 <TabularCPD representing P(fa1:4 | c:2) at 0x10d074748>,
 <TabularCPD representing P(fa2:5 | c:2) at 0x10d13cba8>,
 <TabularCPD representing P(fa3:3 | c:2) at 0x1a14f2b898>,
 <TabularCPD representing P(fa4:5 | c:2) at 0x10c8cf710>,
 <TabularCPD representing P(fa5:4 | c:2) at 0x10d08c630>,
 <TabularCPD representing P(fa6:4 | c:2) at 0x10d08cc88>,
 <TabularCPD representing P(fa7:4 | c:2) at 0x10d08ccc0>,
 <TabularCPD representing P(fa8:5 | c:2) at 0x10c8c6978>,
 <TabularCPD representing P(fa9:3 | c:2) at 0x10c8c6390>,
 <TabularCPD representing P(fb1:4 | c:2, fa1:4) at 0x10d4be438>,
 <TabularCPD representing P(fb2:5 | c:2, fa2:5) at 0x10d13cc18>,
 <TabularCPD representing P(fb3:3 | c:2, fa3:3) at 0x1a14f2b940>,
 <TabularCPD representing P(fb4:5 | c:2, fa4:5) at 0x10d0898d0>,
 <TabularCPD representing P(fb5:4 | c:2, fa5:4) at 0x1a14f47a58>,
 <TabularCPD representing P(fb6:4 | c:2, fa6:4) at 0x10d089860>,
 <TabularCPD representing P(fb7:4 |

In [48]:
#model1.predict(testSameLabels)
model1.check_model()

True

In [49]:
print(model1.get_cpds('c'))

+------+-----+
| c(0) | 0.5 |
+------+-----+
| c(1) | 0.5 |
+------+-----+


In [50]:
model1.local_independencies('fa1')

(fa1 _|_ fb4, fa9, fa7, fa8, fa2, fa5, fb8, fa3, fb6, fb7, fb3, fb5, fa4, fb2, fb9, fa6 | c)

In [51]:
from pgmpy.inference import VariableElimination

In [52]:
#model2 = BayesianModel([('fa1'-'fb1', 'fa2'-'fb2')])
infer = VariableElimination(model1)

In [53]:
#actual testing starts here
import json as json
truPredSameLabels = 0
truPredDiffLabels = 0
    

for i in range(0, len(testSameLabels)):
    testCase = testSameLabels.iloc[i]
    actual_class = testCase['c']
    del testCase['c']
    evd = json.loads(testCase.to_json())
    predicted_class = (infer.map_query(['c'], evidence= evd)['c'])
    if(actual_class == predicted_class):
        truPredSameLabels = truPredSameLabels+1

print((truPredSameLabels * 100)/len(testSameLabels))


for i in range(0, len(testDiffLabels)):
    testCase = testDiffLabels.iloc[i]
    actual_class = testCase['c']
    del testCase['c']
    evd = json.loads(testCase.to_json())
    predicted_class = (infer.map_query(['c'], evidence= evd)['c'])
    if(actual_class == predicted_class):
        truPredDiffLabels = truPredDiffLabels+1


print((truPredDiffLabels * 100)/len(testDiffLabels))

64.57337883959045


/Users/phaniram/anaconda/lib/python3.6/site-packages/pgmpy-0.1.6-py3.6.egg/pgmpy/factors/discrete/DiscreteFactor.py:377: RuntimeWarning: invalid value encountered in true_divide
  phi.values = phi.values / phi.values.sum()


87.57679180887372
